In [2]:
import pyspark.pandas as ps
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml.feature import Word2Vec
import pyspark.sql.functions as F

# increase max result size
spark = SparkSession.builder.master('local[*]').config("spark.driver.memory", "6g").config("spark.executor.memory", "6g").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","24g").config("spark.driver.maxResultSize", "6g").getOrCreate()

/home/tobias.grantner/.cache/pypoetry/virtualenvs/tu-dopp-ws23-OrQDfWK4-py3.9/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/21 21:21:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/21 21:21:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/21 21:21:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Calculate tf-idf

In [3]:
df = ps.read_parquet('/data/data.parquet', index_col=['reviewerID', 'asin'])
df['text'] = df['summary'] + ' ' + df['reviewText']
df = df[['category', 'overall', 'text']]
df: DataFrame = df.reset_index().to_spark()
df = df.dropna(subset=['text'])

/home/tobias.grantner/.cache/pypoetry/virtualenvs/tu-dopp-ws23-OrQDfWK4-py3.9/lib/python3.9/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


24/01/21 21:14:40 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(df)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [ ]:
rescaledData = rescaledData.drop('text', 'words', 'rawFeatures')
rescaledData.write.parquet('/data/tfidf.parquet')

# Calculate embeddings

In [3]:
df = ps.read_parquet('/data/data.parquet', index_col=['reviewerID', 'asin'])
df['text'] = df['summary'] + ' ' + df['reviewText']
df = df[['category', 'overall', 'text']]
df: DataFrame = df.reset_index().to_spark()
df = df.withColumn('words', F.split(F.col('text'), ' ')).drop('text')
df = df.dropna(subset=['words'])

/home/tobias.grantner/.cache/pypoetry/virtualenvs/tu-dopp-ws23-OrQDfWK4-py3.9/lib/python3.9/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [ ]:
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="words", outputCol="embedding")
model = word2Vec.fit(df)
result = model.transform(df)

In [ ]:
result.write.parquet('/data/embedded.parquet')